# Quarterly Fundamentals Dataset (Local Files Only)

This notebook replicates the Appendix G financial-fundamentals rules using local repository files only (no SQL server).

Inputs:
- `financials_data/quarterly/*_income_statement_quarterly.csv`
- `financials_data/quarterly/*_balance_sheet_quarterly.csv`
- `financials_data/quarterly/*_cash_flow_quarterly.csv`
- `stock_data/*.csv` (for quarter-end close and market-cap features)

Output:
- Quarterly panel by `ticker`, `yr`, `qtr`
- All variables from CTE-like `f` (raw fundamentals + ratios + growth)
- Computed `market_cap`, `log_mktcap`, `book_to_market`


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd


In [ ]:
# Locate project root whether notebook runs from SQL_Queries/ or project root
PROJECT_ROOT = Path.cwd()
if not (PROJECT_ROOT / "financials_data").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

QUARTERLY_DIR = PROJECT_ROOT / "financials_data" / "quarterly"
STOCK_DIR = PROJECT_ROOT / "stock_data"

if not QUARTERLY_DIR.exists():
    raise FileNotFoundError(f"Missing directory: {QUARTERLY_DIR}")
if not STOCK_DIR.exists():
    raise FileNotFoundError(f"Missing directory: {STOCK_DIR}")

QUARTERLY_DIR, STOCK_DIR


In [ ]:
# Required metrics from the Appendix G rules
INCOME_METRICS = {
    "Net Income": "net_income",
    "Total Revenue": "total_revenue",
    "Operating Income": "operating_income",
}

BALANCE_METRICS = {
    "Total Assets": "total_assets",
    "Stockholders Equity": "total_equity",
    "Total Debt": "total_debt",
    "Current Assets": "current_assets",
    "Current Liabilities": "current_liabilities",
    "Cash And Cash Equivalents": "cash_equivalents",
    "Ordinary Shares Number": "shares_outstanding",
}

CASHFLOW_METRICS = {
    "Free Cash Flow": "free_cash_flow",
    "Operating Cash Flow": "operating_cash_flow",
}

def safe_div(numerator, denominator):
    den = denominator.replace(0, np.nan)
    return numerator / den


In [ ]:
def _ticker_from_quarterly_filename(path: Path, suffix: str) -> str:
    name = path.name
    if not name.endswith(suffix):
        raise ValueError(f"Unexpected filename format: {name}")
    return name[: -len(suffix)]

def load_statement_long(files, suffix, required_metric_names):
    frames = []

    for path in files:
        ticker = _ticker_from_quarterly_filename(path, suffix)
        df = pd.read_csv(path)
        metric_col = df.columns[0]

        df = df[df[metric_col].isin(required_metric_names)].copy()
        if df.empty:
            continue

        long_df = df.melt(
            id_vars=[metric_col],
            var_name="report_date",
            value_name="metric_value",
        ).rename(columns={metric_col: "metric_name"})

        long_df["ticker"] = ticker
        long_df["report_date"] = pd.to_datetime(long_df["report_date"], errors="coerce")
        long_df["metric_value"] = pd.to_numeric(long_df["metric_value"], errors="coerce")

        long_df = long_df.dropna(subset=["report_date"]) 
        frames.append(long_df[["ticker", "report_date", "metric_name", "metric_value"]])

    if not frames:
        return pd.DataFrame(columns=["ticker", "report_date", "metric_name", "metric_value"])

    return pd.concat(frames, ignore_index=True)

def pivot_metrics(long_df, metric_map):
    wide = (
        long_df.pivot_table(
            index=["ticker", "report_date"],
            columns="metric_name",
            values="metric_value",
            aggfunc="max",
        )
        .rename(columns=metric_map)
        .reset_index()
    )

    # Ensure expected output columns always exist.
    for col in metric_map.values():
        if col not in wide.columns:
            wide[col] = np.nan

    return wide


In [ ]:
income_files = sorted(QUARTERLY_DIR.glob("*_income_statement_quarterly.csv"))
balance_files = sorted(QUARTERLY_DIR.glob("*_balance_sheet_quarterly.csv"))
cashflow_files = sorted(QUARTERLY_DIR.glob("*_cash_flow_quarterly.csv"))

print(f"Income files: {len(income_files)}")
print(f"Balance files: {len(balance_files)}")
print(f"Cash flow files: {len(cashflow_files)}")

income_long = load_statement_long(
    files=income_files,
    suffix="_income_statement_quarterly.csv",
    required_metric_names=set(INCOME_METRICS.keys()),
)
balance_long = load_statement_long(
    files=balance_files,
    suffix="_balance_sheet_quarterly.csv",
    required_metric_names=set(BALANCE_METRICS.keys()),
)
cashflow_long = load_statement_long(
    files=cashflow_files,
    suffix="_cash_flow_quarterly.csv",
    required_metric_names=set(CASHFLOW_METRICS.keys()),
)

print(income_long.shape, balance_long.shape, cashflow_long.shape)


In [ ]:
# CTE-style tables from Appendix G
# i_q
i_q = pivot_metrics(income_long, INCOME_METRICS)

# b_q
b_q = pivot_metrics(balance_long, BALANCE_METRICS)

# cf_q
cf_q = pivot_metrics(cashflow_long, CASHFLOW_METRICS)

i_q.shape, b_q.shape, cf_q.shape


In [ ]:
# raw_fundamentals = i_q JOIN b_q JOIN cf_q on ticker + report_date
raw_fundamentals = (
    i_q
    .merge(b_q, on=["ticker", "report_date"], how="inner")
    .merge(cf_q, on=["ticker", "report_date"], how="inner")
)

raw_fundamentals["yr"] = raw_fundamentals["report_date"].dt.year
raw_fundamentals["qtr"] = raw_fundamentals["report_date"].dt.quarter

base_cols = [
    "ticker", "report_date", "yr", "qtr",
    "net_income", "total_revenue", "operating_income",
    "total_assets", "total_equity", "total_debt",
    "current_assets", "current_liabilities",
    "cash_equivalents", "shares_outstanding",
    "free_cash_flow", "operating_cash_flow",
]
raw_fundamentals = raw_fundamentals[base_cols].copy()
raw_fundamentals.head()


In [ ]:
# fundamentals CTE ratios
fundamentals = raw_fundamentals.copy()

fundamentals["roe"] = safe_div(fundamentals["net_income"], fundamentals["total_equity"]) * 100
fundamentals["roa"] = safe_div(fundamentals["net_income"], fundamentals["total_assets"]) * 100
fundamentals["op_margin"] = safe_div(fundamentals["operating_income"], fundamentals["total_revenue"]) * 100
fundamentals["debt_to_equity"] = safe_div(fundamentals["total_debt"], fundamentals["total_equity"])
fundamentals["liquidity_ratio"] = safe_div(fundamentals["cash_equivalents"], fundamentals["total_assets"])
fundamentals["current_ratio"] = safe_div(fundamentals["current_assets"], fundamentals["current_liabilities"])
fundamentals["free_cf_margin"] = safe_div(fundamentals["free_cash_flow"], fundamentals["total_revenue"]) * 100
fundamentals["ocf_to_assets"] = safe_div(fundamentals["operating_cash_flow"], fundamentals["total_assets"])

fundamentals.head()


In [ ]:
# f CTE: add lagged revenue + revenue growth by ticker-quarter
f = fundamentals.sort_values(["ticker", "yr", "qtr", "report_date"]).copy()
f["prev_revenue"] = f.groupby("ticker")["total_revenue"].shift(1)
f["revenue_growth"] = safe_div(f["total_revenue"] - f["prev_revenue"], f["prev_revenue"]) * 100

f.head()


In [ ]:
def load_quarter_end_prices(stock_dir: Path):
    frames = []

    for path in sorted(stock_dir.glob("*.csv")):
        ticker = path.stem
        df = pd.read_csv(path, usecols=["Date", "Close"])

        # Handles timezone-aware strings like 2020-01-02 00:00:00-05:00
        dt = pd.to_datetime(df["Date"], errors="coerce", utc=True)
        df["date"] = dt.dt.tz_convert(None)
        df["close_q"] = pd.to_numeric(df["Close"], errors="coerce")

        df = df.dropna(subset=["date"]).sort_values("date")
        df["yr"] = df["date"].dt.year
        df["qtr"] = df["date"].dt.quarter

        q_end = (
            df.groupby(["yr", "qtr"], as_index=False)
            .tail(1)
            .rename(columns={"date": "quarter_end_date"})
        )
        q_end["ticker"] = ticker

        frames.append(q_end[["ticker", "yr", "qtr", "quarter_end_date", "close_q"]])

    if not frames:
        return pd.DataFrame(columns=["ticker", "yr", "qtr", "quarter_end_date", "close_q"])

    return pd.concat(frames, ignore_index=True)

quarter_prices = load_quarter_end_prices(STOCK_DIR)
quarter_prices.head()


In [ ]:
# Final quarterly fundamentals dataset
quarterly_fundamentals = f.merge(
    quarter_prices,
    on=["ticker", "yr", "qtr"],
    how="left",
)

quarterly_fundamentals["market_cap"] = (
    quarterly_fundamentals["close_q"] * quarterly_fundamentals["shares_outstanding"]
)
quarterly_fundamentals["log_mktcap"] = np.where(
    quarterly_fundamentals["market_cap"] > 0,
    np.log(quarterly_fundamentals["market_cap"]),
    np.nan,
)
quarterly_fundamentals["book_to_market"] = safe_div(
    quarterly_fundamentals["total_equity"],
    quarterly_fundamentals["market_cap"],
)

final_cols = [
    "ticker", "yr", "qtr",
    "net_income", "total_revenue", "operating_income",
    "total_assets", "total_equity", "total_debt",
    "current_assets", "current_liabilities",
    "cash_equivalents", "shares_outstanding",
    "free_cash_flow", "operating_cash_flow",
    "roe", "roa", "op_margin",
    "debt_to_equity", "liquidity_ratio", "current_ratio",
    "free_cf_margin", "ocf_to_assets",
    "prev_revenue", "revenue_growth",
    "market_cap", "log_mktcap", "book_to_market",
]

quarterly_fundamentals = (
    quarterly_fundamentals[final_cols]
    .sort_values(["ticker", "yr", "qtr"])
    .reset_index(drop=True)
)

print(quarterly_fundamentals.shape)
quarterly_fundamentals.head()


In [ ]:
output_csv = Path("quarterly_fundamentals_from_local_files.csv")
quarterly_fundamentals.to_csv(output_csv, index=False)
output_csv.resolve()
